This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [2]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json
import pprint

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [4]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
api_url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {
    'api_key': API_KEY,
    'start_date': '2017-01-01',
    'end_date': '2017-01-02',
}
json_response = requests.get(url=api_url, params=params)
json_response.raise_for_status()
response = json.loads(json_response.content.decode('utf-8'))
pprint.pprint(response)

{'dataset_data': {'collapse': None,
                  'column_index': None,
                  'column_names': ['Date',
                                   'Open',
                                   'High',
                                   'Low',
                                   'Close',
                                   'Change',
                                   'Traded Volume',
                                   'Turnover',
                                   'Last Price of the Day',
                                   'Daily Traded Units',
                                   'Daily Turnover'],
                  'data': [['2017-01-02',
                            34.99,
                            35.94,
                            34.99,
                            35.8,
                            None,
                            44700.0,
                            1590561.0,
                            None,
                            None,
                            None]],

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [10]:
# Constants.
CLOSE_COL = 'Close'
DATE_COL = 'Date'
HIGH_COL = 'High'
LOW_COL = 'Low'
OPEN_COL = 'Open'
VOLUME_COL = 'Traded Volume'

# 1. Collect the data.
params = {
    'api_key': API_KEY,
    'start_date': '2017-01-01',
    'end_date': '2018-01-01',
}
json_response = requests.get(url=api_url, params=params)
json_response.raise_for_status()

# 2. Convert the data to a Python dictionary.
response = json.loads(json_response.content.decode('utf-8'))
column_names = response['dataset_data']['column_names']

def to_dict(data_row):
    ans = {}
    for i, column_name in enumerate(column_names):
        ans[column_name] = data_row[i]
    return ans

data = [to_dict(d) for d in response['dataset_data']['data']]

# 3. Find the lowest and highest open, ignoring days that do not have an open.
lowest_open = min((d for d in data if d[OPEN_COL]), key = lambda d: d[OPEN_COL])
highest_open = max((d for d in data if d[OPEN_COL]), key = lambda d: d[OPEN_COL])
days_without_open = [d for d in data if not d[OPEN_COL]]

print('#3: The lowest and highest opens.')
print(f'The lowest recorded open occurred on {lowest_open[DATE_COL]} and was {lowest_open[OPEN_COL]}.')
print(f'The highest recorded open occurred on {highest_open[DATE_COL]} and was {highest_open[OPEN_COL]}.')
print(f'The following days did not have opens recorded: {[d[DATE_COL] for d in days_without_open]}.')
print()

# 4. Largest change in any one day.
def change(d):
    return d[HIGH_COL] - d[LOW_COL]
largest_change = max((d for d in data if d[HIGH_COL] and d[LOW_COL]), key = change)

print(f'#4 The largest recorded change occurred on {largest_change[DATE_COL]} and was {change(largest_change)}.')

# 5. Largest change between any two days.
sorted_by_date = sorted(data, key = lambda k: k[DATE_COL])
i = max(
    # Include consecutive dates that both have recorded closing values.
    (i for i in range(len(sorted_by_date) - 2) if sorted_by_date[i][CLOSE_COL] and sorted_by_date[i+1][CLOSE_COL]),
    key = lambda k: abs(sorted_by_date[i][CLOSE_COL] - sorted_by_date[i+1][CLOSE_COL])
)
value = sorted_by_date[i+1][CLOSE_COL] - sorted_by_date[i][CLOSE_COL]
print(f'#5: The largest change between two days was {value} which occurred between {sorted_by_date[i][DATE_COL]} and {sorted_by_date[i+1][DATE_COL]}.')

# 6. Average trading volume.
trading_volumes = [d[VOLUME_COL] for d in data if d[VOLUME_COL]]
average_trading_volume = sum(trading_volumes) / len(trading_volumes)

print(f'#6: The average traded volume was {average_trading_volume}.')

# 7 Median trading volume.
def median(data):
    s = sorted(data)
    if len(s) % 2 == 1:
        return s[len(s)//2]
    return (s[len(s)//2 - 1] + s[len(s)//2]) / 2

print(f'#7: The median traded volume was {median(trading_volumes)}.')


#3: The lowest and highest opens.
The lowest recorded open occurred on 2017-01-24 and was 34.0.
The highest recorded open occurred on 2017-12-14 and was 53.11.
The following days did not have opens recorded: ['2017-05-01', '2017-04-17', '2017-04-14'].

#4 The largest recorded change occurred on 2017-05-11 and was 2.8100000000000023.
#5: The largest change between two days was -0.3200000000000003 which occurred between 2017-01-02 and 2017-01-03.
#6: The average traded volume was 89124.33725490196.
#7: The median traded volume was 76286.0.
